In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth = 120) 
torch.set_grad_enabled(True) 

from torch.utils.tensorboard import SummaryWriter #allows to send data to tensorboard files 

#use below code for google colab
#%load_ext tensorboard
#%tensorboard --logdir=runs

In [11]:
print(torch.__version__)

1.6.0


In [12]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [13]:
class Network(nn.Module):
        
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)


    def forward(self, t):
        
        # (1) input layer
        t = t
        
        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size = 2, stride =2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size = 2, stride =2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
       
        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t) 

        # (6) ouput layer
        t = self.out(t)
        return t

In [14]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'  
    ,train=True    
    ,download=True 
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [15]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle = True)

# Starting with tensorboard(network graph and images)

In [17]:
tb = SummaryWriter() #run start when the object instance is created 

network = Network()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)  #creates grid of images that we'll ultimately see in tensorboard 

tb.add_image('images', grid)
tb.add_graph(network, images)
tb.close()

In [ ]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images,labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(10):
    
    total_loss = 0
    total_correct = 0
    
#batch = next(iter(train_loader)) # Get a single batch

    for batch in train_loader: #get batch
        images, labels = batch

        preds = network(images) #pass batch
        loss = F.cross_entropy(preds, labels) #calculate loss
    
        optimizer.zero_grad() #pytorch accumulates the grad after each pass of the batch so we make sure it's zero
        loss.backward() #calc gradients 
        optimizer.step() #update weights #each time we pass a batch the weights are updates, so if we have 100 batches, it updates 100 times and takes 100 steps towards the minimum of the loss function

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct/len(train_set), epoch)

    tb.add_histogram('con1.bias',network.conv1.bias,epoch)
    tb.add_histogram('con1.weight',network.conv1.weight,epoch)
    tb.add_histogram('con1.weight.grad',network.conv1.weight.grad,epoch)

    print('epoch :', epoch, 'total_correct:', total_correct, 'loss :', total_loss)